In [1]:
from datetime import datetime
import gzip
import tarfile 

import altair as alt
import dask.dataframe as dd
import numpy as np
import pandas as pd

In [19]:
# Country codes, names, continents
countries = pd.read_csv(
    "https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv"
)

In [20]:
country_income_classification = pd.read_excel('https://databank.worldbank.org/data/download/site-content/CLASS.xlsx', sheet_name='List of economies', usecols=range(4), nrows=218).fillna('Unknown')
class_codes = {
    'High income': 'HIC', 
    'Low income': 'LIC', 
    'Lower middle income': 'LMIC',
    'Upper middle income': 'UMIC', 
    'Unknown': 'UNK'
}
country_income_classification['income_code'] = country_income_classification['Income group'].apply(lambda c: class_codes[c])


In [21]:
first_line_seen = False
total_genomes = 0
with gzip.open('metadata_2022-03-24_22-57.tsv.gz', 'rb') as infile:
    with open('metadata.tsv', 'w') as outfile:
        for line in infile:
            line_str = line.decode('utf')
            if not first_line_seen:
                outfile.write(line_str)
                first_line_seen = True
            fields = line_str.split('\t')
            total_genomes += 1
            if fields[5] == 'Africa':
                outfile.write(line_str)

In [23]:
def handle_two_part_dates(date):
    parts = date.split("-")
    if len(parts) == 2:
        parts.append("1")
    date = "-".join(parts)
    return date
africa_metadata = pd.read_csv('metadata.tsv', delimiter='\t', index_col=0)
africa_metadata.date = africa_metadata.date.apply(handle_two_part_dates)
africa_metadata

,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,region_exposure,country_exposure,...,pango_lineage,GISAID_clade,originating_lab,submitting_lab,authors,url,title,paper_url,date_submitted,purpose_of_sequencing
strain,,,,,,,,,,,,,,,,,,,,,
Algeria/00003/2021,ncov,EPI_ISL_5052208,?,2021-08-01,Africa,Algeria,Ouargla,NaN,Africa,Algeria,...,B.1.617.2,GK,"NIC, Viral Respiratory Unit",Virology Departement,Fayez Khardine et al,?,?,?,2021-10-11,NaN
Algeria/00018/2021,ncov,EPI_ISL_5052207,?,2021-07-30,Africa,Algeria,Touggourt,NaN,Africa,Algeria,...,AY.20,GK,"NIC, Viral Respiratory Unit",Virology Departement,Fayez Khardine et al,?,?,?,2021-10-11,NaN
Algeria/01064/2021,ncov,EPI_ISL_5052202,?,2021-06-15,Africa,Algeria,Sidi Bel Abbes,NaN,Africa,Algeria,...,B.1.1.7,GR,"NIC, Viral Respiratory Unit",Virology Departement,Fayez Khardine et al,?,?,?,2021-10-11,NaN
Algeria/02615/2021,ncov,EPI_ISL_5052210,?,2021-07-11,Africa,Algeria,Sidi Bel Abbes,NaN,Africa,Algeria,...,AY.100,GK,"NIC, Viral Respiratory Unit",Virology Departement,Fayez Khardine et al,?,?,?,2021-10-11,NaN
Algeria/03594/2021,ncov,EPI_ISL_5052209,?,2021-08-14,Africa,Algeria,Biskra,NaN,Africa,Algeria,...,AY.125,G,"NIC, Viral Respiratory Unit",Virology Departement,Fayez Khardine et al,?,?,?,2021-10-11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
senegal/SN-IRMBO-003411/2021,ncov,EPI_ISL_6274669,?,2021-10-21,Africa,Senegal,Dakar,Diamniadio,Africa,Senegal,...,AY.25,GK,Iressef Genomics lab,IRESSEF,Souleymane MBOUP et al,?,?,?,2021-11-15,NaN
senegal/SN-IRVAC-015/2021,ncov,EPI_ISL_6274754,?,2021-07-16,Africa,Senegal,Dakar,Diamniadio,Africa,Senegal,...,None,G,Iressef Genomics lab,IRESSEF,Souleymane MBOUP et al,?,?,?,2021-11-15,NaN
senegal/SN-IRVAC-140-CON/2021,ncov,EPI_ISL_6274744,?,2021-07-18,Africa,Senegal,Dakar,Diamniadio,Africa,Senegal,...,AY.33,GK,Iressef Genomics lab,IRESSEF,Souleymane MBOUP et al,?,?,?,2021-11-15,NaN


In [34]:
#ensure that date is valid before any further processing
def is_date_valid(date):
    try:
        datetime.strptime(date, "%Y-%m-%d")
    except ValueError:
        return False
    return True
    
africa_metadata = africa_metadata[africa_metadata.date.apply(is_date_valid)]

In [42]:
# africa_metadata = gisaid_metadata[gisaid_metadata.region == "Africa"]
# only retain things with good dates
africa_metadata = africa_metadata[
    africa_metadata.apply(lambda x: len(x["date"].split("-")) == 3, axis=1)
]
# drop things without a Nextstrain clade - these are typically poor quality
africa_metadata = africa_metadata[africa_metadata.Nextstrain_clade.notna()]

# fix up a submitting lab names
africa_metadata.loc[
    africa_metadata.submitting_lab
    == "KRISP, KZn Research Innovation and Sequencing Platform",
    "submitting_lab",
] = "KRISP, KZN Research Innovation and Sequencing Platform"
africa_metadata.loc[africa_metadata.submitting_lab.isin(
    ['CERI, Centre for Epidemic Response and Innvoation, Stellenbosch University and KRISP, KZN Research Innovation and Sequencing Platform, UKZN.',
    'CERI, Centre for Epidemic Response and Innovation, Stellenbosch Univeristy & KRISP, KZN Research Innovation and Sequencing Platform',
    'CERI, Centre for Epidemic Response and Innovation, Stellenbosch University and KRISP, KZN Research Innovation and Sequencing Platform, UKZN.'
    'CERI, Centre for Epidemic Response and Innovation, Stellenbosch University and CERI-KRISP, KZN Research Innovation and Sequencing Platform']), 
                    "submitting_lab"] = 'CERI, Centre for Epidemic Response and Innovation'
africa_metadata.loc[africa_metadata.submitting_lab.isin(
    ['National Health Laboratory Service/University of Cape Town (National Health Laboratory Service/University of Cape Town (NHLS/UCT))',
     'National Health Laboratory Service/University of Cape Town (NHLS/UCT)',
     'National Health Laboratory Service/UCT']), "submitting_lab"] = 'NHLS/UCT'
africa_metadata.loc[africa_metadata.submitting_lab.isin(
    ['National Health Laboratory Service (NHLS), Tygerberg',
     'Division of Medical Virology, Stellenbosch University and National Health Laboratory Service (NHLS)',
     'Division of Medical Virology, National Health Laboratory Service (NHLS), Tygerberg Hospital / Stellenbosch University',
     'Stellenbosch University and NHLS',
     'National Health Laboratory Services, Virology']), "submitting_lab"] = 'Division of Medical Virology, Stellenbosch University and NHLS Tygerberg Hospital'
africa_metadata.loc[africa_metadata.submitting_lab == 'ZARV, Department Mdeical Virology, University of Pretoria', 
                    "submitting_lab"] = 'ZARV, Department Medical Virology, University of Pretoria'
africa_metadata.loc[
    africa_metadata.submitting_lab
    == "Where sequence data have been generated and submitted to GISAID",
    "submitting_lab",
] = "MRC/UVRI & LSHTM Uganda Research Unit"
africa_metadata.loc[
    africa_metadata.submitting_lab == "KEMRI-Wellcome Trust Research Programme,Kilifi",
    "submitting_lab",
] = "KEMRI-Wellcome Trust Research Programme/KEMRI-CGMR-C Kilifi"
africa_metadata.loc[
    (africa_metadata.country == "Nigeria")
    & (
        africa_metadata.submitting_lab.str.startswith(
            "African Centre of Excellence for Genomics of Infectious Diseases"
        )
    ),
    "submitting_lab",
] = "ACEGID, African Centre of Excellence for Genomics of Infectious Diseases, Redeemer’s University, Ede"
africa_metadata.loc[
    africa_metadata.submitting_lab == "Redeemer's University, ACEGID", "submitting_lab"
] = "ACEGID, African Centre of Excellence for Genomics of Infectious Diseases, Redeemer’s University, Ede"
africa_metadata.loc[
    (africa_metadata.country == "Nigeria")
    & (africa_metadata.submitting_lab.str.startswith("National")),
    "submitting_lab",
] = "NCDC, National Reference Laboratory, Nigeria Centre for Disease Control, Gaduwa, Abuja, Nigeria"
africa_metadata.loc[
    (africa_metadata.country == "Democratic Republic of the Congo")
    & (
        africa_metadata.submitting_lab
        == "Pathogen Sequencing Lab, National Institute for Biomedical Research (INRB)"
    ),
    "submitting_lab",
] = "INRB, Pathogen Sequencing Lab, National Institute for Biomedical Research"

# add date year / month fields
africa_metadata["date_yearmon"] = africa_metadata.apply(
    lambda r: "-".join(r.date.split("-")[:-1]), axis=1
)
africa_metadata["date_submitted_yearmon"] = africa_metadata.apply(
    lambda r: "-".join(r.date_submitted.split("-")[:-1]), axis=1
)

# calculate the number of days between sample collection and sample submission
africa_metadata["days_to_submit"] = africa_metadata.apply(
    lambda r: int(
        (
            datetime.strptime(r.date_submitted, "%Y-%m-%d")
            - datetime.strptime(r.date, "%Y-%m-%d")
        ).total_seconds()
        // (3600 * 24)
    ),
    axis=1,
)

In [44]:
# make a dict mapping GISAID country name to ISO 2 letter code
name_to_two_letter_code = {
    "Union of the Comoros": "KM",
    "Republic of the Congo": "CG",
    "Côte d'Ivoire": "CI",
    "Democratic Republic of the Congo": "CD",
    "Eswatini": "SZ",
    "Guinea": "GN",
}
for country_name in africa_metadata.country.unique():
    country_info = countries[countries.Country_Name.str.contains(country_name)]
    if len(country_info) == 1:
        name_to_two_letter_code[country_name] = country_info.iloc[
            0
        ].Two_Letter_Country_Code

In [45]:
alt.Chart(
    africa_metadata.groupby(["country", "submitting_lab"])
    .count()
    .reset_index()
    .sort_values("virus", ascending=False)
).mark_bar().encode(
    x=alt.X("country", title="Country", sort="-y"),
    y=alt.Y("sum(virus)", title="Number of Samples"),
    color=alt.Color("submitting_lab", sort="-y"),
    tooltip=["submitting_lab", "sum(virus)"],
)

alt.Chart(...)

In [46]:
alt.Chart(
    africa_metadata.groupby(["date_yearmon", "country"]).count().reset_index(),
    width=600,
).mark_bar().encode(
    x=alt.X("date_yearmon:O", title="Sample Date"),
    y=alt.Y("virus", title="Number of Samples"),
    color=alt.Color("country", sort="-y"),
    tooltip="country",
)

alt.Chart(...)

In [47]:
alt.Chart(
    africa_metadata.groupby(["date_submitted_yearmon", "country"])
    .count()
    .reset_index(),
    width=600,
).mark_bar().encode(
    x=alt.X("date_submitted_yearmon:O", title="Submission Date"),
    y=alt.X("virus", title="Number of Samples"),
    color=alt.Color("country", sort="-y"),
    tooltip="country",
)

alt.Chart(...)

In [48]:
alt.Chart(
    africa_metadata.groupby(["country"]).days_to_submit.mean().reset_index()
).mark_bar().encode(
    x=alt.X("country", title="Country"),
    y=alt.Y("days_to_submit", title="Days to submit sample to GISAID"),
)

alt.Chart(...)

In [70]:
def date_greater_filter(date, cutoff):
    date_parts = date.split('-')
    date_day = int(date_parts[2]) if len(date_parts) == 3 else 0
    date_month = int(date_parts[1]) if len(date_parts) >= 2 else 0
    date_year = int(date_parts[0])
    return [date_year, date_month, date_day] >= cutoff

def date_lesser_filter(date, cutoff):
    return not date_greater_filter(date, cutoff)

data = africa_metadata[africa_metadata.apply(lambda r:date_lesser_filter(r['date_submitted'], [2022,3,1]), axis=1)]
alt.Chart(data.groupby('date_submitted_yearmon').count().reset_index(), title="Number of Genomes Per Month submitted to GISAID from Africa", width=600).mark_line(interpolate="monotone").encode(
    alt.X("date_submitted_yearmon", title="Year / Month"), 
    alt.Y('virus', title="Number of Genomes"))

alt.Chart(...)